In [1]:
#!/usr/bin/env python3
import skeleton_with_bonus
from skeleton_with_bonus import loadRandomDescriptors,loadRandomDescriptors_v3, getFiles, dictionary,assignments,vlad,generate_multiple_codebooks_v2,multi_vlad,generate_multiple_codebooks,apply_pca,distances,evaluate, train_svm, exemplar_classification, exemplar_classification_parallel,computeDescs,getFiles_v2,loadRandomDescriptors_v2,compute_and_save_desc
import os
import _pickle as cPickle
import gzip
from tqdm import tqdm
import numpy as np
import cv2
import multiprocessing
import matplotlib.pyplot as plt

In [2]:
in_train = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/train'
suffix = '_SIFT_patch_pr.pkl.gz'
labels_train = 'icdar17_labels_train.txt'

In [3]:
files_train, labels_train = getFiles(in_train, suffix, labels_train)
print('#train: {}'.format(len(files_train)))

#train: 1182


In [4]:
descriptors = loadRandomDescriptors(files_train, max_descriptors=500000)

100%|██████████| 100/100 [00:01<00:00, 77.00it/s]


In [5]:
if not os.path.exists('mus.pkl.gz'):
    descriptors = loadRandomDescriptors(files_train, max_descriptors=500000)
    print('> loaded {} descriptors:'.format(len(descriptors)))

    # cluster centers
    print('> compute dictionary')
    mus = dictionary(descriptors, n_clusters=100)
    with gzip.open('mus.pkl.gz', 'wb') as fOut:
        cPickle.dump(mus, fOut, -1)
else:
    with gzip.open('mus.pkl.gz', 'rb') as f:
        mus = cPickle.load(f)
print(mus.shape)

(100, 64)


In [ ]:
gamma = 1
gmp = False
in_test = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/test'
suffix = '_SIFT_patch_pr.pkl.gz'
labels_test = 'icdar17_labels_test.txt'
powernorm = True

print('> compute VLAD for test')
files_test, labels_test = getFiles(in_test, suffix,
                                    labels_test)
print('#test: {}'.format(len(files_test)))
fname = 'enc_test_gmp{}.pkl.gz'.format(gamma) if gmp else 'enc_test.pkl.gz'
if not os.path.exists(fname):
    enc_test = vlad(files_test, mus, powernorm,gamma,gmp)
    with gzip.open(fname, 'wb') as fOut:
        cPickle.dump(enc_test, fOut, -1)
else:
    with gzip.open(fname, 'rb') as f:
        enc_test = cPickle.load(f)
        
print('> evaluate')
evaluate(enc_test, labels_test)

> compute VLAD for test
#test: 3600
> evaluate
Top-1 accuracy: 0.8197222222222222 - mAP: 0.6283903541081468


In [ ]:

# Parameters for VLAD computation
n_clusters = 100  # Example number of clusters for k-means
max_descriptors = 500000  # Example max descriptors for MiniBatchKMeans
C = 1000  # Regularization parameter for SVM
gamma=900
powernorm=True
gmp=False

# Paths for input data
in_train = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/train'
suffix = '_SIFT_patch_pr.pkl.gz'  # Suffix for your training files
labels_train = 'icdar17_labels_train.txt'

# Compute or load the VLAD encodings for training data
enc_train_file = 'enc_train_gmp_{}.pkl.gz'
if not os.path.exists(enc_train_file):
    print('> compute VLAD for train')
    files_train, labels_train = getFiles(in_train, suffix, labels_train)
    print('#train: {}'.format(len(files_train)))
    enc_train = vlad(files_train, mus, powernorm,gamma,gmp)
    with gzip.open(enc_train_file, 'wb') as fOut:
        cPickle.dump(enc_train, fOut, -1)
else:
    with gzip.open(enc_train_file, 'rb') as f:
        enc_train = cPickle.load(f)


> compute VLAD for train
#train: 1182


100%|██████████| 1182/1182 [00:21<00:00, 56.00it/s]


In [ ]:
# Evaluate VLAD encodings
print('> evaluate VLAD encodings')
evaluate(enc_train, labels_train)


> evaluate VLAD encodings
Top-1 accuracy: 0.8211111111111111 - mAP: 0.6303366667827087


In [ ]:
# With parallelized
print('> compute E-SVM for test')
enc_train = np.array(enc_train) if isinstance(enc_train, list) else enc_train
enc_test = np.array(enc_test) if isinstance(enc_test, list) else enc_test
new_encs_test = exemplar_classification_parallel(enc_train, enc_test,C_value=1000, n_jobs=-1)


> compute E-SVM for test


In [ ]:
# Optionally, evaluate the new E-SVM encodings
print('> evaluate E-SVM encodings')
evaluate(new_encs_test, labels_test)

> evaluate E-SVM encodings
Top-1 accuracy: 0.8852777777777778 - mAP: 0.7526401045072001


## e - Extracting own features

In [ ]:
if __name__ == "__main__":
    image_directory = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/icdar2017-training-binary'
    new_directory = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/icdar2017-training-binary/train_new'
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)

    # Prepare a list of arguments for the function
    file_paths = [(os.path.join(image_directory, f), new_directory) 
                  for f in os.listdir(image_directory) 
                  if f.endswith(('.jpg', '.png'))]

    # Use multiprocessing to process files in parallel
    with multiprocessing.Pool() as pool:
        pool.starmap(compute_and_save_desc, file_paths)

    print("Descriptors for all images have been computed and saved in the new directory.")


Descriptors for all images have been computed and saved in the new directory.


In [ ]:
if __name__ == "__main__":
    image_directory = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/ScriptNet-HistoricalWI-2017-binarized'
    new_directory = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/ScriptNet-HistoricalWI-2017-binarized/test_new'
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)

    # Prepare a list of arguments for the function
    file_paths = [(os.path.join(image_directory, f), new_directory) 
                  for f in os.listdir(image_directory) 
                  if f.endswith(('.jpg', '.png'))]

    # Use multiprocessing to process files in parallel
    with multiprocessing.Pool() as pool:
        pool.starmap(compute_and_save_desc, file_paths)

    print("Descriptors for all images have been computed and saved in the new directory.")


In [ ]:
in_train='/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/icdar2017-training-binary/train_new'
suffix='.desc'
labels_train = 'icdar17_labels_train.txt'

In [ ]:
files_train, labels_train = getFiles_v2(in_train, suffix, labels_train)
print('#train: {}'.format(len(files_train)))


#train: 1182


In [ ]:
descriptors = loadRandomDescriptors_v2(files_train, max_descriptors=500000)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:01<00:00, 53.76it/s]


In [ ]:
descriptors.shape

(488415, 128)

In [ ]:
if not os.path.exists('mus_new.pkl.gz'):
    descriptors = loadRandomDescriptors(files_train, max_descriptors=500000)
    print('> loaded {} descriptors:'.format(len(descriptors)))

    # cluster centers
    print('> compute dictionary')
    mus_new = dictionary(descriptors, n_clusters=100)
    with gzip.open('mus_new.pkl.gz', 'wb') as fOut:
        cPickle.dump(mus_new, fOut, -1)
else:
    with gzip.open('mus_new.pkl.gz', 'rb') as f:
        mus_new = cPickle.load(f)
print(mus_new.shape)

(100, 128)


In [ ]:
gamma = 1
gmp = False
in_test = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/ScriptNet-HistoricalWI-2017-binarized/test_new'
suffix = '.desc'
labels_test = 'icdar17_labels_test.txt'
powernorm = False

print('> compute VLAD for test')
files_test, labels_test = getFiles(in_test, suffix,
                                    labels_test)
print('#test: {}'.format(len(files_test)))
fname = 'enc_test_new_gmp{}.pkl.gz'.format(gamma) if gmp else 'enc_test_new.pkl.gz'
if not os.path.exists(fname):
    enc_test_new = vlad(files_test, mus_new, powernorm)
    with gzip.open(fname, 'wb') as fOut:
        cPickle.dump(enc_test_new, fOut, -1)
else:
    with gzip.open(fname, 'rb') as f:
        enc_test_new = cPickle.load(f)
        
print('> evaluate')
evaluate(enc_test_new, labels_test)

> compute VLAD for test
#test: 3600


100%|██████████| 3600/3600 [01:37<00:00, 36.91it/s]


> evaluate
Top-1 accuracy: 0.8227777777777778 - mAP: 0.6329760064760036


In [ ]:

# Parameters for VLAD computation
n_clusters = 100  # Example number of clusters for k-means
max_descriptors = 500000  # Example max descriptors for MiniBatchKMeans
C = 1000  # Regularization parameter for SVM

# Paths for input data
in_train = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/icdar2017-training-binary/train_new'
suffix = '.desc'  # Suffix for your training files
labels_train = 'icdar17_labels_train.txt'

# Compute or load the VLAD encodings for training data
enc_train_file = 'enc_train_new.pkl.gz'
if not os.path.exists(enc_train_file):
    print('> compute VLAD for train')
    files_train, labels_train = getFiles(in_train, suffix, labels_train)
    print('#train: {}'.format(len(files_train)))
    enc_train_new = vlad(files_train, mus_new, powernorm)
    with gzip.open(enc_train_file, 'wb') as fOut:
        cPickle.dump(enc_train_new, fOut, -1)
else:
    with gzip.open(enc_train_file, 'rb') as f:
        enc_train_new = cPickle.load(f)


> compute VLAD for train
#train: 1182


  0%|          | 0/1182 [00:00<?, ?it/s]

100%|██████████| 1182/1182 [00:31<00:00, 37.41it/s]


In [ ]:
# With parallelized
print('> compute E-SVM for test')
enc_train_new = np.array(enc_train_new) if isinstance(enc_train_new, list) else enc_train_new
enc_test_new = np.array(enc_test_new) if isinstance(enc_test_new, list) else enc_test_new
new_encs_test = exemplar_classification_parallel(enc_train_new, enc_test_new,C_value=1000, n_jobs=-1)
# Optionally, evaluate the new E-SVM encodings
print('> evaluate E-SVM encodings')
evaluate(new_encs_test, labels_test)


> compute E-SVM for test
> evaluate E-SVM encodings
Top-1 accuracy: 0.8813888888888889 - mAP: 0.744585942886534


# f - Generalized Max Pooling

In [ ]:
gamma = 900
gmp = True
powernorm = True
in_test = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/ScriptNet-HistoricalWI-2017-binarized/test_new'
suffix = '.desc'
labels_test = 'icdar17_labels_test.txt'



print('> compute VLAD for test')
files_test, labels_test = getFiles(in_test, suffix,
                                    labels_test)
print('#test: {}'.format(len(files_test)))
fname = 'enc_test_new_gmp{}.pkl.gz'.format(gamma) if gmp else 'enc_test_new.pkl.gz'
if not os.path.exists(fname):
    enc_test_new = vlad(files_test, mus_new,powernorm, gamma,gmp)
    with gzip.open(fname, 'wb') as fOut:
        cPickle.dump(enc_test_new, fOut, -1)
else:
    with gzip.open(fname, 'rb') as f:
        enc_test_new = cPickle.load(f)
        
print('> evaluate')
evaluate(enc_test_new, labels_test)

> compute VLAD for test
#test: 3600


100%|██████████| 3600/3600 [07:37<00:00,  7.86it/s]


> evaluate
Top-1 accuracy: 0.835 - mAP: 0.6450024517439594


In [ ]:
gamma = 1
gmp = True
powernorm = True
in_test = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/ScriptNet-HistoricalWI-2017-binarized/test_new'
suffix = '.desc'
labels_test = 'icdar17_labels_test.txt'



print('> compute VLAD for test')
files_test, labels_test = getFiles(in_test, suffix,
                                    labels_test)
print('#test: {}'.format(len(files_test)))
fname = 'enc_test_new_gmp{}.pkl.gz'.format(gamma) if gmp else 'enc_test_new.pkl.gz'
if not os.path.exists(fname):
    enc_test_new = vlad(files_test, mus_new,powernorm, gamma,gmp)
    with gzip.open(fname, 'wb') as fOut:
        cPickle.dump(enc_test_new, fOut, -1)
else:
    with gzip.open(fname, 'rb') as f:
        enc_test_new = cPickle.load(f)
        
print('> evaluate')
evaluate(enc_test_new, labels_test)

> compute VLAD for test
#test: 3600


100%|██████████| 3600/3600 [07:05<00:00,  8.47it/s]


> evaluate
Top-1 accuracy: 0.8380555555555556 - mAP: 0.650423440945756


In [ ]:
# Parameters for VLAD computation
n_clusters = 100  # Example number of clusters for k-means
max_descriptors = 500000  # Example max descriptors for MiniBatchKMeans
C = 1000  # Regularization parameter for SVM

# Paths for input data
in_train = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/icdar2017-training-binary/train_new'
suffix = '.desc'  # Suffix for your training files
labels_train = 'icdar17_labels_train.txt'
gmp=True
powernorm=False
gamma=1

# Compute or load the VLAD encodings for training data
enc_train_file = 'enc_train_new_gmp{}.pkl.gz'.format(gamma)
if not os.path.exists(enc_train_file):
    print('> compute VLAD for train')
    files_train, labels_train = getFiles(in_train, suffix, labels_train)
    print('#train: {}'.format(len(files_train)))
    enc_train_new = vlad(files_train, mus_new, powernorm,gamma,gmp)
    with gzip.open(enc_train_file, 'wb') as fOut:
        cPickle.dump(enc_train_new, fOut, -1)
else:
    with gzip.open(enc_train_file, 'rb') as f:
        enc_train_new = cPickle.load(f)


> compute VLAD for train
#train: 1182


100%|██████████| 1182/1182 [02:26<00:00,  8.08it/s]


In [ ]:
# With parallelized
print('> compute E-SVM for test')
enc_train_new = np.array(enc_train_new) if isinstance(enc_train_new, list) else enc_train_new
enc_test_new = np.array(enc_test_new) if isinstance(enc_test_new, list) else enc_test_new
new_encs_test = exemplar_classification_parallel(enc_train_new, enc_test_new,C_value=1000, n_jobs=-1)
# Optionally, evaluate the new E-SVM encodings
print('> evaluate E-SVM encodings')
evaluate(new_encs_test, labels_test)


> compute E-SVM for test
> evaluate E-SVM encodings
Top-1 accuracy: 0.9033333333333333 - mAP: 0.7775841934326508


# g - PCA whitening and multi-VLAD

In [7]:
# Main code
gamma = 1
gmp = True
powernorm = True
max_descriptors = 50000  # Example value for max descriptors
num_clusters = 100  # Number of clusters for k-means
num_codebooks = 5  # Number of different codebooks
seeds = [0, 1, 2, 3, 4]

in_test = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/ScriptNet-HistoricalWI-2017-binarized/test_new'
suffix = '.desc'
labels_test = 'icdar17_labels_test.txt'

in_train = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/Computer Vision/Week 3/icdar2017-training-binary/train_new'
suffix = '.desc'  # Suffix for your training files
labels_train = 'icdar17_labels_train.txt'

print('> Retrieve files and labels for test dataset')
files_test, labels_test = getFiles_v2(in_test, suffix, labels_test)
print('#test: {}'.format(len(files_test)))

print('> Retrieve files and labels for train dataset')
files_train, labels_train = getFiles_v2(in_train, suffix, labels_train)

print('#train: {}'.format(len(files_train)))
# Generate multiple codebooks using TRAINING data
print('> Generating multiple codebooks from training data')
codebooks = generate_multiple_codebooks_v2(files_train, num_clusters, num_codebooks, max_descriptors, seeds)

# Compute Multi-VLAD for TRAINING data
print('> Compute Multi-VLAD for training dataset')
multi_vlad_encodings_train = []
for mus in codebooks:
    enc_train = vlad(files_train, mus, powernorm, gamma, gmp)
    multi_vlad_encodings_train.append(enc_train)
concatenated_encodings_train = np.concatenate(multi_vlad_encodings_train, axis=1)

# Train PCA on TRAINING data encodings
print('> Apply PCA on training concatenated encodings')
reduced_enc_train, pca_model = apply_pca(concatenated_encodings_train)

# Compute Multi-VLAD for TEST data using the same codebooks
print('> Compute Multi-VLAD for test dataset')
multi_vlad_encodings_test = []
for mus in codebooks:
    enc_test = vlad(files_test, mus, powernorm, gamma, gmp)
    multi_vlad_encodings_test.append(enc_test)
concatenated_encodings_test = np.concatenate(multi_vlad_encodings_test, axis=1)

# Apply the SAME PCA model to TEST data encodings
reduced_enc_test_new = pca_model.transform(concatenated_encodings_test)

# Save and/or Evaluate the reduced TEST encodings
fname = 'multi_enc_test_new_gmp{}_pca.pkl.gz'.format(gamma) if gmp else 'multi_enc_test_new_pca.pkl.gz'
if not os.path.exists(fname):
    print('> Saving reduced test encodings')
    with gzip.open(fname, 'wb') as fOut:
        cPickle.dump(reduced_enc_test_new, fOut, -1)
else:
    print('> Loading previously saved reduced test encodings')
    with gzip.open(fname, 'rb') as f:
        reduced_enc_test_new = cPickle.load(f)

print('> Evaluate')
evaluate(reduced_enc_test_new, labels_test)


> Retrieve files and labels for test dataset
#test: 3600
> Retrieve files and labels for train dataset
#train: 1182
> Generating multiple codebooks from training data
Generating codebook 1/5 with seed 0...


100%|██████████| 100/100 [00:02<00:00, 46.39it/s]


Generating codebook 2/5 with seed 1...


100%|██████████| 100/100 [00:02<00:00, 44.26it/s]


Generating codebook 3/5 with seed 2...


...

> Compute Multi-VLAD for training dataset


100%|██████████| 1182/1182 [02:26<00:00,  8.09it/s]


> Apply PCA on training concatenated encodings
> Compute Multi-VLAD for test dataset


100%|██████████| 3600/3600 [06:52<00:00,  8.72it/s]


> Loading previously saved reduced test encodings
> Evaluate
Top-1 accuracy: 0.8958333333333334 - mAP: 0.7729058808820068


In [8]:
# With parallelized
print('> compute E-SVM for test')
enc_train_new = np.array(reduced_enc_train) if isinstance(reduced_enc_train, list) else reduced_enc_train
enc_test_new = np.array(reduced_enc_test_new) if isinstance(reduced_enc_test_new, list) else reduced_enc_test_new
new_encs_test = exemplar_classification_parallel(enc_train_new, enc_test_new,C_value=1000, n_jobs=-1)
# Optionally, evaluate the new E-SVM encodings
print('> evaluate E-SVM encodings')
evaluate(new_encs_test, labels_test)


> compute E-SVM for test
> evaluate E-SVM encodings
Top-1 accuracy: 0.8313888888888888 - mAP: 0.6537229927617099
